In [ ]:
# INSTALLATION / MISE À JOUR COMFYUI STANDARD & CLEAN

import os
import subprocess
import sys

COMFY_PATH = "/workspace/ComfyUI"

print("🚀 Nettoyage et Installation Standard de ComfyUI...")

if os.path.exists(COMFY_PATH):
    print("📂 Dossier détecté. Réinitialisation vers la version officielle...")
    # Suppression des fichiers custom qui pourraient polluer le démarrage (comme cuda_malloc.py ou dossiers app modifiés)
    # On garde les dossiers importants (models, input, output, custom_nodes)
    subprocess.run(f"cd {COMFY_PATH} && git remote set-url origin https://github.com/comfyanonymous/ComfyUI.git", shell=True)
    subprocess.run(f"cd {COMFY_PATH} && git fetch origin && git reset --hard origin/master", shell=True)
    # Nettoyage des fichiers non suivis par git (ignore les modèles et nodes)
    subprocess.run(f"cd {COMFY_PATH} && git clean -fd --exclude=models --exclude=custom_nodes --exclude=input --exclude=output", shell=True)
    print("✅ Reset terminé.")
else:
    print("📥 Première installation (Clonage officiel)...")
    subprocess.run(f"git clone https://github.com/comfyanonymous/ComfyUI.git {COMFY_PATH}", shell=True)

# INSTALLATION DES REQUIREMENTS OFFICIELS
print("\n📦 Installation des dépendances officielles (requirements.txt)...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pip"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", f"{COMFY_PATH}/requirements.txt"])

# AJOUT DES DÉPENDANCES MANQUANTES DÉTECTÉES PRÉCÉDEMMENT
print("\n🔧 Ajout des modules complémentaires indispensables (SQLAlchemy, etc.)...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "sqlalchemy", "alembic", "pydantic-settings", "sentencepiece", "einops"])

print("\n🎉 Installation standard terminée et prête.")

In [ ]:
# === INSTALLATION TOTALE FLUX (Nodes + Modèles Vision + Dépendances + Fix NSFW) ===
# FIX: Ajout de l'installation système aria2 et fallback sur wget

import os
import subprocess
import sys

COMFY_DIR = "/workspace/ComfyUI"
CUSTOM_NODES_DIR = f"{COMFY_DIR}/custom_nodes"

print("🚀 DÉMARRAGE DE L'INSTALLATION FLUX & TOOLS...")

# --- 0. PRÉ-REQUIS SYSTÈME (FIX POUR ARIA2C) ---
print("🛠️ Vérification des outils système...")
try:
    # On tente d'installer aria2 silencieusement
    subprocess.run(["apt-get", "update"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    subprocess.run(["apt-get", "install", "-y", "aria2"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    print("   ✅ Aria2 installé.")
except Exception as e:
    print("   ⚠️ Impossible d'installer aria2 via apt (pas grave, on utilisera wget).")

# --- 1. LISTE DES NODES (Pack Image Pure) ---
nodes_to_install = [
    "https://github.com/Comfy-Org/ComfyUI-Manager",             
    "https://github.com/rgthree/rgthree-comfy",                 
    "https://github.com/kijai/ComfyUI-KJNodes",                 
    "https://github.com/cubiq/ComfyUI_essentials",              
    "https://github.com/negaga53/comfyui-imgloader",            
    "https://github.com/pythongosssss/ComfyUI-Custom-Scripts",  
    "https://github.com/sonnybox/ComfyUI-SuperNodes",           
    "https://github.com/ltdrdata/ComfyUI-Impact-Pack",          
    "https://github.com/ltdrdata/ComfyUI-Impact-Subpack",       
    "https://github.com/numz/ComfyUI-SeedVR2_VideoUpscaler",    
    "https://github.com/ClownsharkBatwing/RES4LYF",             
    "https://github.com/giriss/comfy-image-saver",              
    "https://github.com/Gourieff/ComfyUI-ReActor",              
    "https://github.com/ltdrdata/was-node-suite-comfyui",       
    "https://github.com/caleboleary/Comfyui-calbenodes", 
    "https://github.com/lldacing/ComfyUI_PuLID_Flux_ll",
    "https://github.com/Jonseed/ComfyUI-Detail-Daemon",
]

# --- 2. INSTALLATION DES NODES ---
# Pré-installation InsightFace (Critique pour ReActor)
try:
    import insightface
except ImportError:
    print("📦 Installation de InsightFace & ONNX...")
    subprocess.run([sys.executable, "-m", "pip", "install", "insightface", "onnxruntime-gpu"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

for repo_url in nodes_to_install:
    node_name = repo_url.split("/")[-1]
    node_path = os.path.join(CUSTOM_NODES_DIR, node_name)
    
    if not os.path.exists(node_path):
        print(f"⬇️ Clonage de : {node_name}...")
        try:
            subprocess.run(["git", "clone", repo_url, node_path], check=True, stdout=subprocess.DEVNULL)
            req_path = os.path.join(node_path, "requirements.txt")
            if os.path.exists(req_path):
                print(f"   📦 Dépendances pour {node_name}...")
                subprocess.run([sys.executable, "-m", "pip", "install", "-r", req_path], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        except subprocess.CalledProcessError:
            print(f"   ❌ Erreur clone {node_name}")
    else:
        print(f"⏩ [SKIP] {node_name} ok.")

# --- 3. TÉLÉCHARGEMENT MODÈLES ULTRALYTICS (YOLO - Pour Impact Pack) ---
print("\n👁️ Vérification des modèles de détection (YOLO)...")
base_yolo = f"{COMFY_DIR}/models/ultralytics"
bbox_path = os.path.join(base_yolo, "bbox")
segm_path = os.path.join(base_yolo, "segm")
os.makedirs(bbox_path, exist_ok=True)
os.makedirs(segm_path, exist_ok=True)

yolo_models = [
    # BBOX (Carrés)
    ("face_yolov8m.pt", "https://huggingface.co/Bingsu/adetailer/resolve/main/face_yolov8m.pt", bbox_path),
    ("hand_yolov8s.pt", "https://huggingface.co/Bingsu/adetailer/resolve/main/hand_yolov8s.pt", bbox_path),
    # SEGM (Masques précis)
    ("person_yolov8m-seg.pt", "https://huggingface.co/Bingsu/adetailer/resolve/main/person_yolov8m-seg.pt", segm_path),
    ("yolov8n-seg.pt", "https://huggingface.co/Bingsu/adetailer/resolve/main/yolov8n-seg.pt", segm_path),
    ("face_yolov8m-seg_60.pt", "https://huggingface.co/bobmin/face_yolov8m-seg_60/resolve/main/face_yolov8m-seg_60.pt", segm_path)
]

for name, url, dest in yolo_models:
    final_path = os.path.join(dest, name)
    if not os.path.exists(final_path):
        print(f"   ⬇️ Téléchargement : {name}")
        # Méthode robuste : Essaie Aria2c, sinon Wget
        try:
            subprocess.run(["aria2c", "-x", "16", "-s", "16", "-k", "1M", "-d", dest, "-o", name, url], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        except (FileNotFoundError, subprocess.CalledProcessError):
            print(f"      ⚠️ Aria2c échoué/absent, utilisation de wget...")
            subprocess.run(["wget", "-q", "-O", final_path, url])

# --- 4. PATCH REACTOR NSFW (AUTOMATIQUE) ---
print("\n🔓 Application du patch ReActor (NSFW Bypass)...")
reactor_path = f"{CUSTOM_NODES_DIR}/ComfyUI-ReActor/scripts/reactor_sfw.py"
bypass_code = """
from PIL import Image
import logging
import os
from scripts.reactor_logger import logger
MODEL_EXISTS = True
def ensure_nsfw_model(nsfwdet_model_path): return True
def nsfw_image(img_data, model_path: str): return False
"""

if os.path.exists(os.path.dirname(reactor_path)):
    try:
        with open(reactor_path, "w") as f: f.write(bypass_code)
        print("   ✅ Patch appliqué sur 'reactor_sfw.py'.")
    except Exception as e:
        print(f"   ❌ Erreur patch : {e}")
else:
    print("   ⚠️ ReActor non trouvé, patch ignoré.")

print("\n🎉 ENVIRONNEMENT FLUX COMPLET ET PRÊT !")
print(">>> Relancez la cellule de lancement (ComfyUI) pour charger tout ça.")

In [ ]:
# FLUX FILES DOWNLOAD (Updated with Flux.2 Klein)

import os
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- 1. CONFIGURATION SÉCURISÉE ---
CIVITAI_TOKEN = os.environ.get("CIVITAI_TOKEN", "")
HF_TOKEN = os.environ.get("HF_TOKEN", "")

# Installation silencieuse de aria2
os.system("apt-get update -y > /dev/null && apt-get install -y aria2 > /dev/null")

# --- 2. BASE DE DONNÉES (Flux Core & LoRAs - Chemins Absolus) ---
db_models = {
    "🚀 Flux.2 Klein (Distilled Models)": [
        ("Flux.2 Klein 9B (Standard)", "https://huggingface.co/black-forest-labs/FLUX.2-klein-9B/resolve/main/flux-2-klein-9b.safetensors?download=true", "/workspace/ComfyUI/models/unet", "flux-2-klein-9b.safetensors", "hf"),
        ("Flux.2 Klein 9B (Base)", "https://huggingface.co/black-forest-labs/FLUX.2-klein-base-9B/resolve/main/flux-2-klein-base-9b.safetensors?download=true", "/workspace/ComfyUI/models/unet", "flux-2-klein-base-9b.safetensors", "hf"),
        ("Flux.2 Klein 4B (Lightweight)", "https://huggingface.co/black-forest-labs/FLUX.2-klein-4B/resolve/main/flux-2-klein-4b.safetensors?download=true", "/workspace/ComfyUI/models/unet", "flux-2-klein-4b.safetensors", "hf"),
    ],
    "🌊 Flux.1 Core (UNET & Checkpoints)": [
        ("Flux.1 Krea Dev (FP8 Scaled)", "https://civitai.com/api/download/models/2079678?type=Model&format=SafeTensor&size=pruned&fp=fp8", "/workspace/ComfyUI/models/unet", "flux1KreaDevFP8_fp8SCALED.safetensors", "civitai"),
        ("Flux.1 Dev Officiel (FP8)", "https://huggingface.co/Comfy-Org/FLUX.1-dev-ComfyUI-Repackaged/resolve/main/flux1-dev-fp8.safetensors?download=true", "/workspace/ComfyUI/models/unet", "flux1-dev-fp8.safetensors", "hf"),
    ],
    "⚙️ Flux Dépendances (VAE & CLIP)": [
        ("VAE Flux (ae.safetensors)", "https://huggingface.co/black-forest-labs/FLUX.1-schnell/resolve/main/ae.safetensors?download=true", "/workspace/ComfyUI/models/vae", "ae.safetensors", "hf"),
        ("T5 XXL (FP8) - Encodeur Texte", "https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp8_e4m3fn.safetensors?download=true", "/workspace/ComfyUI/models/clip", "t5xxl_fp8_e4m3fn.safetensors", "hf"),
        ("CLIP L - Encodeur Texte", "https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors?download=true", "/workspace/ComfyUI/models/clip", "clip_l.safetensors", "hf"),
    ],
    "💄 Flux LoRAs (Body & Styles)": [
        ("FC Flux Perfect Busts", "https://civitai.com/api/download/models/1782533?type=Model&format=SafeTensor", "/workspace/ComfyUI/models/loras", "FC Flux Perfect Busts.safetensors", "civitai"),
        ("Busty Curvy", "https://civitai.com/api/download/models/1732014?type=Model&format=SafeTensor", "/workspace/ComfyUI/models/loras", "busty-curvy.safetensors", "civitai"),
        ("TKTT Steep Slope Tiddies XXL", "https://civitai.com/api/download/models/746019?type=Model&format=SafeTensor", "/workspace/ComfyUI/models/loras", "TKTT_SteepSlopeTiddiesXXL_FLUX.safetensors", "civitai"),
        ("Venus Body (Thicc)", "https://civitai.com/api/download/models/2244224?type=Model&format=SafeTensor", "/workspace/ComfyUI/models/loras", "venus_body_thicc.safetensors", "civitai"),
        ("Olivia 2025 (Privé)", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/olivia2025.safetensors?download=true", "/workspace/ComfyUI/models/loras", "olivia2025.safetensors", "hf"),
    ]
}

# --- 3. INTERFACE GRAPHIQUE ---
style = {'description_width': 'initial'}
layout = widgets.Layout(width='98%')
checkboxes = {}
output_log = widgets.Output(layout={'border': '1px solid #444', 'height': '200px', 'overflow_y': 'scroll'})

accordion_children = []
titles = []

for category, items in db_models.items():
    vbox_items = []
    select_all = widgets.Button(description="Tout cocher", icon="check-square-o", button_style='info', layout=widgets.Layout(width='150px'))
    
    def on_select_all(b, cat=category):
        for k, val in checkboxes.items():
            if k.startswith(cat):
                val['cb'].value = True
    
    select_all.on_click(on_select_all)
    vbox_items.append(select_all)
    
    for item in items:
        name, url, path, filename, src = item
        key = f"{category}|{filename}"
        cb = widgets.Checkbox(value=False, description=name, style=style, layout=layout)
        checkboxes[key] = {'cb': cb, 'data': item}
        vbox_items.append(cb)
        
    accordion_children.append(widgets.VBox(vbox_items))
    titles.append(category)

accordion = widgets.Accordion(children=accordion_children)
for i, title in enumerate(titles):
    accordion.set_title(i, title)
accordion.selected_index = 0

btn_download = widgets.Button(
    description='LANCER LE TÉLÉCHARGEMENT (Flux)',
    button_style='success', 
    layout=widgets.Layout(width='100%', height='60px', margin='20px 0px 0px 0px'),
    icon='download'
)

# --- 4. LOGIQUE DE TÉLÉCHARGEMENT ---
def run_download(b):
    output_log.clear_output()
    with output_log:
        print("🚀 Traitement des fichiers Flux...\n")
        count = 0
        for key, val in checkboxes.items():
            if val['cb'].value:
                count += 1
                item = val['data']
                name, url, path, filename, src = item
                
                os.makedirs(path, exist_ok=True)
                full_path = os.path.join(path, filename)
                
                if os.path.exists(full_path) and os.path.getsize(full_path) > 1024:
                    print(f"⏩ [SKIP] {name} est déjà là.")
                    continue
                
                auth = ""
                final_url = url
                if src == "hf":
                    auth = f'--header="Authorization: Bearer {HF_TOKEN}"'
                elif src == "civitai":
                    sep = "&" if "?" in url else "?"
                    final_url = f"{url}{sep}token={CIVITAI_TOKEN}"
                
                print(f"⬇️ Téléchargement de : {name}")
                cmd = f'aria2c -x 16 -s 16 -k 1M -c {auth} -d "{path}" -o "{filename}" "{final_url}"'
                
                ret = os.system(cmd)
                if ret == 0:
                    print(f"   ✅ Terminé.")
                else:
                    print(f"   ⚠️ Erreur (Code {ret}). Vérifiez votre HF_TOKEN pour les modèles Klein.")
                    
        if count == 0:
            print("⚠️ Aucune case cochée.")
        else:
            print("\n🎉 Opérations terminées.")

btn_download.on_click(run_download)

display(widgets.VBox([
    widgets.Label("🌊 INSTALLATION FLUX.1 & FLUX.2 KLEIN"),
    accordion,
    btn_download,
    output_log
]))

In [ ]:
# ==========================================
# CELLULE FLUX & PULID : CHIRURGIE DE GUERRE
# ==========================================
import os, subprocess, sys

COMFY_DIR = "/workspace/ComfyUI"
PULID_PATH = f"{COMFY_DIR}/custom_nodes/ComfyUI_PuLID_Flux_ll/pulidflux.py"

print("⚔️ [1/4] NETTOYAGE RADICAL & FIX PIP...")
# On force le lien pip et on nettoie l'environnement
subprocess.run("ln -sf /usr/local/bin/pip /usr/bin/pip", shell=True)
wipe = ["torch", "torchvision", "torchaudio", "numpy", "facenet-pytorch", "facexlib", "comfy-kitchen", "comfy-aimdo"]
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y"] + wipe)

print("\n🚀 [2/4] INSTALLATION DU COEUR (Torch 2.5.1)...")
# Installation de la version dont ComfyUI a besoin pour fonctionner
subprocess.check_call([sys.executable, "-m", "pip", "install", "--no-cache-dir",
    "torch==2.5.1+cu124", "torchvision==0.20.1+cu124", "torchaudio==2.5.1+cu124", 
    "--extra-index-url", "https://download.pytorch.org/whl/cu124"])

print("\n💉 [3/4] INSTALLATION DES NODES (Mode Anti-Sabotage)...")
# A. On installe d'abord les librairies "toxiques" SANS leurs dépendances pour les empêcher de downgrade Torch
toxiques = ["facenet-pytorch", "facexlib", "insightface"]
subprocess.check_call([sys.executable, "-m", "pip", "install", "--no-deps"] + toxiques)

# B. On installe tout le reste normalement
le_reste = [
    "numpy==1.26.4", "opencv-contrib-python", "sqlalchemy", "alembic", 
    "comfy-aimdo", "comfy-kitchen", "onnxruntime-gpu", "segment-anything", 
    "ultralytics", "dill", "sageattention", "open-clip-torch", "diffusers", 
    "rotary-embedding-torch", "GitPython", "toml", "omegaconf", "PyWavelets"
]
subprocess.check_call([sys.executable, "-m", "pip", "install", "--no-cache-dir"] + le_reste)

print("\n🔧 [4/4] PATCHES & LANCEMENT (RTX 4090)...")
# Patch PuLID
if os.path.exists(PULID_PATH):
    with open(PULID_PATH, 'r', encoding='utf-8') as f:
        content = f.read()
    if "def pulid_outer_sample_wrappers_with_override" in content and "**kwargs" not in content:
        content = content.replace("):", ", **kwargs):")
        with open(PULID_PATH, 'w', encoding='utf-8') as f: f.write(content)
        print("✅ PuLID patché.")

# AntelopeV2
antelope_dir = f"{COMFY_DIR}/models/insightface/models/antelopev2"
os.makedirs(antelope_dir, exist_ok=True)
for f in ["1k3d68.onnx", "2d106det.onnx", "genderage.onnx", "glintr100.onnx", "scrfd_10g_bnkps.onnx"]:
    if not os.path.exists(f"{antelope_dir}/{f}"):
        subprocess.run(f"aria2c -x 16 -s 16 -k 1M -c https://huggingface.co/okaris/antelopev2/resolve/main/{f} -d {antelope_dir} -o {f}", shell=True, stdout=subprocess.DEVNULL)

# Injection finale du PATH pour le Manager
os.environ["PATH"] = "/usr/local/bin:" + os.environ["PATH"]
os.chdir(COMFY_DIR)

!python main.py --listen 0.0.0.0 --port 8188 --disable-auto-launch \
    --normalvram --bf16-unet --use-sage-attention

⚔️ [1/4] NETTOYAGE RADICAL & FIX PIP...
Found existing installation: torch 2.2.2
Uninstalling torch-2.2.2:
  Successfully uninstalled torch-2.2.2
Found existing installation: torchvision 0.17.2
Uninstalling torchvision-0.17.2:
  Successfully uninstalled torchvision-0.17.2
Found existing installation: torchaudio 2.5.1+cu124
Uninstalling torchaudio-2.5.1+cu124:
  Successfully uninstalled torchaudio-2.5.1+cu124
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: facenet-pytorch 2.6.0
Uninstalling facenet-pytorch-2.6.0:
  Successfully uninstalled facenet-pytorch-2.6.0
Found existing installation: facexlib 0.3.0
Uninstalling facexlib-0.3.0:
  Successfully uninstalled facexlib-0.3.0
Found existing installation: comfy-kitchen 0.2.7
Uninstalling comfy-kitchen-0.2.7:
  Successfully uninstalled comfy-kitchen-0.2.7
Found existing installation: comfy-aimdo 0.1.8
Uninstalling comfy-aimdo-0.1.8:
  Successfully unin

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 MB 691.0 MB/s  0:00:01a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 453.0 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 1.2 GB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 465.5 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 1.2 GB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 442.5 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 638.6 MB/s  0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 639.0 MB/s  0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 759.7 MB/s  0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 442.4 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 817.2 MB/s  0:00:00
     ━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.63.1 requires numpy<2.4,>=1.22, but you have numpy 2.4.2 which is incompatible.



💉 [3/4] INSTALLATION DES NODES (Mode Anti-Sabotage)...
  Using cached facenet_pytorch-2.6.0-py3-none-any.whl.metadata (12 kB)
  Using cached facexlib-0.3.0-py3-none-any.whl.metadata (4.6 kB)
Using cached facenet_pytorch-2.6.0-py3-none-any.whl (1.9 MB)
Using cached facexlib-0.3.0-py3-none-any.whl (59 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [facenet-pytorch]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 100.1 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 683.1/683.1 kB 1.4 GB/s  0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.4.2
    Uninstalling numpy-2.4.2:
      Successfully uninstalled numpy-2.4.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [comfy-aimdo]


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
facenet-pytorch 2.6.0 requires torch<2.3.0,>=2.2.0, but you have torch 2.5.1+cu124 which is incompatible.
facenet-pytorch 2.6.0 requires torchvision<0.18.0,>=0.17.0, but you have torchvision 0.20.1+cu124 which is incompatible.



🔧 [4/4] PATCHES & LANCEMENT (RTX 4090)...
[START] Security scan
[ComfyUI-Manager] Neither `python -m pip` nor `uv` are available. Cannot proceed with package operations.
Failed to execute startup-script: /workspace/ComfyUI/custom_nodes/ComfyUI-Manager/prestartup_script.py / Neither `pip` nor `uv` are available for package management

Prestartup times for custom nodes:
   0.0 seconds: /workspace/ComfyUI/custom_nodes/rgthree-comfy
   0.3 seconds (PRESTARTUP FAILED): /workspace/ComfyUI/custom_nodes/ComfyUI-Manager

Found comfy_kitchen backend cuda: {'available': True, 'disabled': True, 'unavailable_reason': None, 'capabilities': ['apply_rope', 'apply_rope1', 'dequantize_nvfp4', 'dequantize_per_tensor_fp8', 'quantize_nvfp4', 'quantize_per_tensor_fp8', 'scaled_mm_nvfp4']}
Found comfy_kitchen backend eager: {'available': True, 'disabled': False, 'unavailable_reason': None, 'capabilities': ['apply_rope', 'apply_rope1', 'dequantize_nvfp4', 'dequantize_per_tensor_fp8', 'quantize_nvfp4', 'quant